In [1]:
%load_ext autoreload
%autoreload 2
    
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys


sys.path.append("../")
import vuong_tests10 as vuong_tests_fast
from vuong_test_base import *

In [2]:
class JointNormal1(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([params[0], 0.0], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())
    
    
class JointNormal2(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([0.0, params[0]], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())


def setup_shi(yn,xn):
    # model 1 grad, etc.
    nobs = yn.shape[0]
    model1_param = np.array([yn.mean()])
    model2_param = np.array([xn.mean()])
    
    model1_deriv = JointNormal1(yn,xn)
    ll1 = model1_deriv.loglikeobs(model1_param)
    grad1 =  model1_deriv.score_obs(model1_param).reshape( (nobs,1) )
    hess1 = model1_deriv.hessian(model1_param)
    
    
    model2_deriv = JointNormal2(yn,xn)
    ll2 = model2_deriv.loglikeobs(model2_param)
    grad2 =  model2_deriv.score_obs(model2_param).reshape( (nobs,1) )  
    hess2 = model2_deriv.hessian(model2_param)
    
    return ll1,grad1,hess1,model1_param,ll2,grad2,hess2,model2_param

def gen_data(beta= 1.5, nobs=1000):
    #np.random.seed(1)
    cov = [[25, 0], [0, 1]]
    data = np.random.multivariate_normal([beta,beta], [[25,0],[0,1]],  nobs)
    return data[:,0],data[:,1],nobs

def gen_data3(beta= 1.5, nobs=1000):
    #np.random.seed(1)
    cov = [[25, 0], [0, 1]]
    data = np.random.multivariate_normal([0,beta], [[25,0],[0,1]],  nobs)
    return data[:,0],data[:,1],nobs


yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn)
print(grad1.shape,hess1.shape)
#NOTE! Weird size distortions with shi's test when theta = .5....

(1000, 1) (1, 1)


In [3]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi2(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


def gen_data2(nobs=1000, a=0.25, num_params=4):
    #np.random.seed(1)
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs


In [4]:
# 1. Check variances and correlation of ll1 and ll2
def check_ll_variances(ll1, ll2):
    """Checks variances, covariance, and correlation between ll1 and ll2,
    as well as Var(ll1-ll2) identity."""
    var1 = float(np.var(ll1, ddof=0))
    var2 = float(np.var(ll2, ddof=0))
    cov12 = float(np.mean((ll1 - ll1.mean()) * (ll2 - ll2.mean())))
    var_diff = float(np.var(ll1 - ll2, ddof=0))
    rhs = var1 + var2 - 2.0 * cov12
    corr = cov12 / (np.sqrt(var1) * np.sqrt(var2) + 1e-24)
    print("---- ll variance/cov check ----")
    print(f"Var(ll1)     = {var1:.6g}")
    print(f"Var(ll2)     = {var2:.6g}")
    print(f"Cov(ll1,ll2) = {cov12:.6g}, Corr = {corr:.6g}")
    print(f"Var(ll1-ll2) = {var_diff:.6g}, Var(ll1)+Var(ll2)-2Cov = {rhs:.6g}")
    print("--------------------------------")

# 2. Variance identity check for your studentizer formula
def check_variance_identity(ll1, ll2, epsilon):
    """Checks the correct identity for Var(√n · d̃):
    0.5·Var(m_even) + 0.5·Var(m_odd) equals the paper's formula."""
    n = ll1.shape[0]
    idx_even = np.arange(0, n, 2)
    idx_odd  = np.arange(1, n, 2)
    
    # Group-specific contributions
    m_even = (ll1[idx_even] - ll2[idx_even]) + epsilon * ll1[idx_even]
    m_odd  = (ll1[idx_odd]  - ll2[idx_odd])  - epsilon * ll2[idx_odd]
    
    var_even = float(np.var(m_even, ddof=0)) if idx_even.size > 1 else 0.0
    var_odd  = float(np.var(m_odd,  ddof=0)) if idx_odd.size  > 1 else 0.0
    lhs = 0.5 * (var_even + var_odd)
    
    # Paper's formula
    sigma2  = float(np.var(ll1 - ll2, ddof=0))
    sigmaA2 = float(np.var(ll1[idx_even], ddof=0)) if idx_even.size > 1 else 0.0
    sigmaB2 = float(np.var(ll2[idx_odd],  ddof=0)) if idx_odd.size  > 1 else 0.0
    rhs = (1 + epsilon) * sigma2 + (epsilon**2 / 2.0) * (sigmaA2 + sigmaB2)
    
    print("---- variance identity check (correct) ----")
    print(f"0.5·Var(m_even)+0.5·Var(m_odd) = {lhs:.6g}")
    print(f"(1+eps)·Var(diff)+eps^2/2·(VarA+VarB) = {rhs:.6g}")
    print(f"abs diff = {abs(lhs - rhs):.6g}")
    print("-------------------------------------------")

# 3. Sandwich H/V checks
def hv_sanity(grad, hess, ridge=1e-8):
    """Check H condition number, eigenvalues, and trace(H^-1 V)."""
    H = -np.mean(hess, axis=0)
    H = 0.5*(H + H.T) + ridge * np.eye(H.shape[0])
    V = (grad.T @ grad) / grad.shape[0]
    V = 0.5*(V + V.T)
    eigs = np.linalg.eigvalsh(H)
    cond = np.linalg.cond(H)
    trHinvV = float(np.trace(np.linalg.solve(H, V)))
    print("---- H/V sanity ----")
    print(f"H shape = {H.shape}, cond(H) = {cond:.3g}, eig min/max = {eigs.min():.6g}/{eigs.max():.6g}")
    print(f"tr(H^-1 V) = {trHinvV:.6g}")
    print("--------------------")


def hv_scale_diagnostics(grad, hess, ridge=1e-8, name=""):
    """
    Diagnose H,V scaling for either per-observation Hessians (n,p,p) or a single full-sample Hessian (p,p).
    Prints eigen info and tr(H^{-1} V) for the two plausible scalings in the 2D case.
    """
    n, p = grad.shape
    
    def sym(A): return 0.5 * (A + A.T)
    
    # V_hat: average score cross-product using log-likelihood scores
    V_mean = sym((grad.T @ grad) / n)

    # Single Hessian matrix: try both interpretations
    H2        = sym(-hess)
    H2_divn   = sym(-hess / n)

    H2_r      = H2 + ridge * np.eye(p)
    H2_divn_r = H2_divn + ridge * np.eye(p)

    eig_2     = np.linalg.eigvalsh(H2_r)
    eig_2divn = np.linalg.eigvalsh(H2_divn_r)

    def tr_HinvV(H): return float(np.trace(np.linalg.solve(H, V_mean)))

    tr2        = tr_HinvV(H2_r)
    tr2_divn   = tr_HinvV(H2_divn_r)

    print("Detected 2D Hessian; trying both scalings:")
    print(f"H2 eig min/max       = {eig_2.min():.6g} / {eig_2.max():.6g}")
    print(f"H2/n eig min/max     = {eig_2divn.min():.6g} / {eig_2divn.max():.6g}")
    print(f"tr(H2^-1 V)          = {tr2:.6g}")
    print(f"tr((H2/n)^-1 V)      = {tr2_divn:.6g}")
    print("Target magnitude for tr(H^{-1}V) ≈ parameter dimension p")

print("---------------------------------------")

---------------------------------------


In [5]:
import numpy as np
from scipy.stats import norm


def _estimate_HV(grad, hess_total, ridge=1e-8):
    """
    grad: (n,p) per-observation score of the log-likelihood
    hess_total: (p,p) full-sample Hessian of the log-likelihood (sum over i)
    Returns:
    H_hat = -E[∂² log f]  (per-observation average curvature)
    V_hat = E[s s′]       (uncentered score second moment)
    """
    n = grad.shape[0]
    H_hat = -hess_total / n
    S = grad - grad.mean() # i think its supposed to be centered....
    V_hat = (S.T @ S) / n
    return H_hat, V_hat

def _trace_HinvV(H, V):
    return float(np.trace(np.linalg.solve(H, V)))


def diagnostic_optimal_epsilon(
    ll1, grad1, hess1, params1,
    ll2, grad2, hess2, params2,
    alpha=0.05, ridge=1e-8, min_epsilon=1e-6, max_epsilon=10.0,
    floor_mult=None, enforce_positive_cpl=False, verbose=True
):
    nobs = ll1.shape[0]
    idx_even = np.arange(0, nobs, 2)
    idx_odd  = np.arange(1, nobs, 2)

    # Per-sample variances
    sigma2_hat   = float(np.var(ll1 - ll2, ddof=0))
    sigmaA2_hat  = float(np.var(ll1[idx_even], ddof=0)) if idx_even.size > 1 else 0.0
    sigmaB2_hat  = float(np.var(ll2[idx_odd],  ddof=0)) if idx_odd.size  > 1 else 0.0
    sigma_hat    = np.sqrt(max(sigma2_hat, 1e-12))
    diff_term    = sigma2_hat - 2.0*(sigmaA2_hat + sigmaB2_hat)

    H1, V1 = _estimate_HV(grad1, hess1)
    H2, V2 = _estimate_HV(grad2, hess2)
    tr1 = abs(_trace_HinvV(H1, V1))
    tr2 = abs(_trace_HinvV(H2, V2))
    tr_max = max(tr1, tr2)

    # Condition numbers for H1, H2
    condH1 = np.linalg.cond(H1)
    condH2 = np.linalg.cond(H2)

    # Constants for formula
    z = norm.ppf(1 - alpha/2.0)
    phi_z = norm.pdf(z)
    delta_star = (sigma_hat / 2.) * (z - np.sqrt(4. + z**2))
    phi_arg = z - (delta_star / max(sigma_hat, 1e-12))
    phi_term = norm.pdf(phi_arg)
    denom_PL = 4. * (sigma_hat**3 + 1e-24)
    denom_SD = np.sqrt(max((sigmaA2_hat + sigmaB2_hat)/2., 1e-24))

    CPL_num = (abs(delta_star) * abs(diff_term) if enforce_positive_cpl
               else delta_star * diff_term)
    C_PL_hat = phi_term * CPL_num / denom_PL
    C_SD_hat = 2. * phi_z * tr_max / denom_SD

    # Log-log term for rate
    lnln_term = max(np.log(np.log(max(nobs, 3))), 1e-6)
    eps_rate = nobs**(-1/6.) * (lnln_term**(1/3.))

    valid = np.isfinite(C_PL_hat) and np.isfinite(C_SD_hat) and (C_PL_hat > 0) and (C_SD_hat > 0)
    if valid:
        eps_unclipped = ((C_SD_hat / C_PL_hat) ** (1/3.)) * eps_rate
    else:
        eps_unclipped = eps_rate

    # Optional rate-aware floor
    if floor_mult is not None:
        eps_unclipped = max(eps_unclipped, floor_mult * eps_rate)
    eps_final = float(np.clip(eps_unclipped, min_epsilon, max_epsilon))

    # Diagnostics print
    if verbose:
        print("\n---- Diagnostic for Optimal Epsilon ----")
        print(f"n = {nobs}, alpha = {alpha}, z = {z:.4f}")
        print(f"sigma2 = {sigma2_hat:.4g}, sigmaA2 = {sigmaA2_hat:.4g}, sigmaB2 = {sigmaB2_hat:.4g}")
        print(f"sigma = {sigma_hat:.4g}, delta_star = {delta_star:.4g}, phi_term = {phi_term:.4g}")
        print(f"diff_term = sigma2 - 2*(sigmaA2+sigmaB2) = {diff_term:.4g}")
        print(f"H1 cond = {condH1:.2g}, H2 cond = {condH2:.2g}")
        print(f"tr(H1^(-1)V1) = {tr1:.4g}, tr(H2^(-1)V2) = {tr2:.4g}, tr_max = {tr_max:.4g},denom_SD:{denom_SD:.4g}")
        print(f"CPL_num = {CPL_num:.4g}, denom_PL={denom_PL:.4g}, phi_term={phi_term:.4g}")
        print(f"C_SD_hat = {C_SD_hat:.4g}, C_PL_hat = {C_PL_hat:.4g}")
        print(f"lnln_term = {lnln_term:.4g}, eps_rate = {eps_rate:.4g}")
        print(f"valid ratio = {valid}, epsilon raw = {eps_unclipped:.4g}, clipped = {eps_final:.4g}")
        print("----------------------------------------\n")

    return {
        "epsilon": eps_final,
        "n": nobs,
        "sigma2": sigma2_hat,
        "sigmaA2": sigmaA2_hat,
        "sigmaB2": sigmaB2_hat,
        "sigma": sigma_hat,
        "delta_star": delta_star,
        "phi_term": phi_term,
        "diff_term": diff_term,
        "tr1": tr1,
        "tr2": tr2,
        "tr_max": tr_max,
        "condH1": condH1,
        "condH2": condH2,
        "C_SD_hat": C_SD_hat,
        "C_PL_hat": C_PL_hat,
        "lnln_term": lnln_term,
        "eps_rate": eps_rate,
        "eps_unclipped": eps_unclipped,
        "eps_clipped": eps_final,
        "valid": valid
    }

yn,xn,nobs = gen_data(nobs=250,beta=0)
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn)
check_ll_variances(ll1, ll2)
check_variance_identity(ll1, ll2, epsilon=0.5)  # or try your computed epsilon
hv_scale_diagnostics(grad1, hess1, name="Model A")
hv_scale_diagnostics(grad2, hess2, name="Model B")
hv_sanity(grad1, hess1)
hv_sanity(grad2, hess2)
diagnostic_optimal_epsilon(ll1, grad1, hess1, params1, ll2, grad2, hess2, params2, alpha=0.05)

yn,xn,nobs = gen_data2(nobs=250)
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi2(yn,xn)
diagnostic_optimal_epsilon(ll1, grad1, hess1, params1, ll2, grad2, hess2, params2, alpha=0.05)

yn,xn,nobs = gen_data3(nobs=250)
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn)
diagnostic_optimal_epsilon(ll1, grad1, hess1, params1, ll2, grad2, hess2, params2, alpha=0.05)

---- ll variance/cov check ----
Var(ll1)     = 328.319
Var(ll2)     = 332.565
Cov(ll1,ll2) = 329.95, Corr = 0.998533
Var(ll1-ll2) = 0.983305, Var(ll1)+Var(ll2)-2Cov = 0.983305
--------------------------------
---- variance identity check (correct) ----
0.5·Var(m_even)+0.5·Var(m_odd) = 83.8036
(1+eps)·Var(diff)+eps^2/2·(VarA+VarB) = 84.0201
abs diff = 0.216471
-------------------------------------------
Detected 2D Hessian; trying both scalings:
H2 eig min/max       = 250 / 250
H2/n eig min/max     = 1 / 1
tr(H2^-1 V)          = 0.103648
tr((H2/n)^-1 V)      = 25.912
Target magnitude for tr(H^{-1}V) ≈ parameter dimension p
Detected 2D Hessian; trying both scalings:
H2 eig min/max       = 250 / 250
H2/n eig min/max     = 1 / 1
tr(H2^-1 V)          = 0.00450489
tr((H2/n)^-1 V)      = 1.12622
Target magnitude for tr(H^{-1}V) ≈ parameter dimension p
---- H/V sanity ----
H shape = (1, 1), cond(H) = 1, eig min/max = 250/250
tr(H^-1 V) = 0.103648
--------------------
---- H/V sanity ----
H sha

{'epsilon': 0.24309022778997746,
 'n': 250,
 'sigma2': 3.2136844993342235,
 'sigmaA2': 418.12714735112735,
 'sigmaB2': 345.1319707869925,
 'sigma': np.float64(1.7926752353212847),
 'delta_star': np.float64(-0.7531893692471859),
 'phi_term': np.float64(0.023484710981724093),
 'diff_term': -1523.3045517769056,
 'tr1': 25.97620624253868,
 'tr2': 0.9932060522868483,
 'tr_max': 25.97620624253868,
 'condH1': np.float64(1.0),
 'condH2': np.float64(1.0),
 'C_SD_hat': np.float64(0.1554291952283472),
 'C_PL_hat': np.float64(1.1692605384799049),
 'lnln_term': np.float64(1.7086424843059957),
 'eps_rate': np.float64(0.4763144227414109),
 'eps_unclipped': np.float64(0.24309022778997746),
 'eps_clipped': 0.24309022778997746,
 'valid': np.True_}